## 📖 Parte 1: Introduzione agli Indirizzi IP

### Cos'è un indirizzo IP?

Un indirizzo IPv4 è un numero a **32 bit** usato per identificare dispositivi su una rete.

**Rappresentazioni dello stesso IP (192.168.1.1):**

```
1. Dotted Decimal (umani):    "192.168.1.1"
2. Binario (32 bit):           11000000.10101000.00000001.00000001
3. Esadecimale:                0xC0A80101
4. Decimale:                   3232235777
5. Array di ottetti:           [192, 168, 1, 1]
```

### Perché servono multiple rappresentazioni?

- **Stringa**: leggibile per gli utenti → input/output
- **uint32_t**: operazioni matematiche e bitwise veloci → elaborazione
- **Array ottetti**: accesso diretto ai byte → parsing e conversioni

### Informazioni associate a un IP

Oltre alla rappresentazione, un IP ha:
- **Classe** (A, B, C, D, E) - determinata dal primo ottetto
- **Tipo** (Riservato, Privato, Pubblico) - determinato da range specifici
- **Netmask** - quanti bit sono per la rete vs host
- **Network address** - primo IP della subnet
- **Broadcast address** - ultimo IP della subnet

---

## 🔧 Parte 2: Tipi a Larghezza Fissa

### Il Problema con i Tipi Standard

```c
int x;        // Quanti bit? 16? 32? 64? Dipende dal compilatore!
char c;       // È signed o unsigned? Dipende!
```

**Problema**: su sistemi diversi, `int` può essere 16, 32 o 64 bit.

### La Soluzione: stdint.h

`<stdint.h>` (C99+) fornisce tipi con **larghezza garantita**:

```c
uint8_t   →  unsigned  8 bit (0 to 255)
uint16_t  →  unsigned 16 bit (0 to 65535)
uint32_t  →  unsigned 32 bit (0 to 4294967295)
uint64_t  →  unsigned 64 bit

int8_t    →  signed  8 bit (-128 to 127)
int16_t   →  signed 16 bit
int32_t   →  signed 32 bit
int64_t   →  signed 64 bit
```

### Perché per gli IP usiamo uint32_t?

- IPv4 = **esattamente 32 bit**
- **Unsigned**: nessun IP è "negativo"
- Operazioni bitwise più intuitive senza segno
- Portabilità: funziona su qualsiasi architettura

### Perché uint8_t per gli ottetti?

- Ogni ottetto è **esattamente 8 bit** (0-255)
- Risparmio memoria: 4 byte esatti per 4 ottetti
- Type safety: non possiamo assegnare valori > 255

---

## 🎨 Parte 3: Enumerazioni - Valori con Significato

### Il Problema con le Costanti

```c
// Approccio 1: Magic numbers (MALE!)
if (type == 0) { ... }  // Cosa significa 0?
if (type == 1) { ... }  // E 1?

// Approccio 2: Define (meglio, ma...)
#define IP_RESERVED 0
#define IP_PRIVATE 1
#define IP_PUBLIC 2
int type = IP_RESERVED;
type = 999;  // Nessun errore! Ma 999 non è valido!
```

### La Soluzione: Enum

```c
typedef enum {
    IP_RESERVED,   // = 0 (automatico)
    IP_PRIVATE,    // = 1
    IP_PUBLIC      // = 2
} IPType;

IPType type = IP_RESERVED;  // Tipo sicuro!
type = 999;  // Warning del compilatore!
```

### Vantaggi

1. **Leggibilità**: `IP_PRIVATE` dice più di `1`
2. **Type safety**: il compilatore avvisa per valori non validi
3. **Autocomplete**: l'IDE suggerisce i valori possibili
4. **Manutenibilità**: modificare un valore non rompe il codice

### Assegnazione Automatica

```c
enum Color {
    RED,      // 0
    GREEN,    // 1
    BLUE = 10,// 10
    YELLOW    // 11 (continua da 10)
};
```

---

## 🏗️ Parte 4: Struttura IPAddress - Design e Rationale

### Requisiti della Struttura

La struttura `IPAddress` deve contenere:
1. **Multiple rappresentazioni** dell'IP (per efficienza)
2. **Informazioni di rete** (netmask, network, broadcast)
3. **Classificazione** (classe, tipo)
4. **Metadati** (validità)

### Design Principle: Single Source of Truth

```c
// MALE: duplicazione e possibile inconsistenza
char ip_string[16];
uint32_t ip_number;
uint8_t octets[4];
// Se modifico ip_string, devo ricordarmi di aggiornare gli altri!

// BENE: tutto in una struttura
typedef struct {
    uint32_t ip_num;      // Source of truth principale
    char ip_str[16];      // Derivato da ip_num
    uint8_t octets[4];    // Derivato da ip_num
    // ... altre info ...
} IPAddress;
```

### Anatomia della Struttura IPAddress

```
┌─────────────────────────────────────────────────────────────┐
│                    STRUTTURA IPAddress                      │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  SEZIONE 1: Rappresentazione dell'IP                        │
│  ─────────────────────────────────────────────────────      │
│  uint32_t ip_num         [4 byte]  ◄── Master copy          │
│  char ip_str[16]         [16 byte] ◄── Per output           │
│  uint8_t octets[4]       [4 byte]  ◄── Accesso diretto      │
│                                                             │
│  SEZIONE 2: Informazioni di Rete                            │
│  ─────────────────────────────────────────────────────      │
│  uint32_t netmask        [4 byte]  ◄── Es. 0xFFFFFF00       │
│  uint8_t cidr_prefix     [1 byte]  ◄── Es. 24               │
│  uint32_t network_addr   [4 byte]  ◄── Es. 192.168.1.0      │
│  uint32_t broadcast_addr [4 byte]  ◄── Es. 192.168.1.255    │
│                                                             │
│  SEZIONE 3: Classificazione                                 │
│  ─────────────────────────────────────────────────────      │
│  char class              [1 byte]  ◄── 'A', 'B', 'C', ...   │
│  IPType type             [4 byte]  ◄── Enum                 │
│                                                             │
│  SEZIONE 4: Metadati                                        │
│  ─────────────────────────────────────────────────────      │
│  bool is_valid           [1 byte]  ◄── Flag validità        │
│                                                             │
│  TOTALE: ~47 byte (senza padding)                           │
└─────────────────────────────────────────────────────────────┘
```

### Scelte di Design e Motivazioni

#### 1. Perché salvare ip_str[16]?

**Pro**: Output immediato senza conversione  
**Contro**: Usa 16 byte  
**Decisione**: Vale la pena per performance di output

Calcolo dimensione:
```
"255.255.255.255\0" = 15 caratteri + 1 null terminator = 16 byte
```

#### 2. Perché salvare octets[4]?

**Pro**: Accesso O(1) a singoli ottetti  
**Contro**: Duplicazione di ip_num  
**Decisione**: Comodo per algoritmi che operano per ottetto

#### 3. Perché cidr_prefix E netmask?

**Ragione**: Conversione frequente tra i due formati  
Esempio: `/24` ↔ `255.255.255.0`

#### 4. Perché bool is_valid?

**Pattern**: Sentinel value per indicare errori  
Permette di ritornare una struct invece di NULL

---

## 💾 Parte 5: Layout di Memoria e Padding

### Allineamento in Memoria

Il compilatore **allinea** i campi delle struct per efficienza:

```c
struct Example {
    char a;      // 1 byte
    int b;       // 4 byte (su x86-64)
    char c;      // 1 byte
};
```

**Layout reale in memoria (x86-64):**

```
┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│ a  │ pad│ pad│ pad│  b  (4 byte int)   │ c  │ pad│ pad│ pad│
└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘
 0    1    2    3    4    5    6    7    8    9   10   11

sizeof(struct Example) = 12 byte (non 6!)
```

### Perché il Padding?

**Ragione**: La CPU legge memoria in "blocchi" allineati (4/8 byte su x86-64)

- Lettura non allineata = 2+ accessi memoria = lento
- Lettura allineata = 1 accesso = veloce

### Come Minimizzare il Padding?

**Regola**: Ordina i campi dal più grande al più piccolo

```c
// MALE (spreca spazio)
struct Bad {
    char a;      // 1 + 3 pad
    int b;       // 4
    char c;      // 1 + 3 pad
    int d;       // 4
};  // = 16 byte

// BENE (ottimizzato)
struct Good {
    int b;       // 4
    int d;       // 4
    char a;      // 1
    char c;      // 1 + 2 pad
};  // = 12 byte
```

### IPAddress Layout Analisi

```c
typedef struct {
    uint32_t ip_num;              // 4 byte, allineato a 4
    char ip_str[16];              // 16 byte
    uint8_t octets[4];            // 4 byte
    uint32_t netmask;             // 4 byte, allineato a 4
    uint8_t cidr_prefix;          // 1 byte
    // [3 byte padding qui]
    uint32_t network_addr;        // 4 byte, allineato a 4
    uint32_t broadcast_addr;      // 4 byte
    char class;                   // 1 byte
    // [3 byte padding]
    IPType type;                  // 4 byte (enum = int)
    bool is_valid;                // 1 byte
    // [3 byte padding finale]
} IPAddress;
// sizeof(IPAddress) ≈ 52-56 byte (dipende dal compilatore)
```

**Nota**: Il padding esatto dipende dal compilatore e dall'architettura.

---

## 📝 Parte 6: Implementazione - Creiamo ip_types.h

Ora implementeremo il file header completo con tutte le definizioni.

### Header Guards

Prima di tutto, ogni header deve avere **header guards** per prevenire inclusioni multiple:

```c
#ifndef IP_TYPES_H    // Se non definito
#define IP_TYPES_H    // Definiscilo

// ... contenuto del file ...

#endif // IP_TYPES_H  // Fine guard
```

**Problema che risolve:**
```c
// file1.h include ip_types.h
// file2.h include ip_types.h
// main.c include file1.h e file2.h
// Senza guards: ip_types.h incluso 2 volte → errore duplicazione!
// Con guards: la seconda inclusione è ignorata → OK
```

### Include Necessari

```c
#include <stdint.h>   // Per uint32_t, uint8_t
#include <stdbool.h>  // Per bool, true, false (C99+)
```

---

In [ ]:
%%bash
# Creiamo la directory di lavoro per questo notebook
mkdir -p /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/01
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/01

# Creiamo ip_types.h completo
cat > ip_types.h << 'EOF'
#ifndef IP_TYPES_H
#define IP_TYPES_H

#include <stdint.h>
#include <stdbool.h>

/**
 * Enumerazione per il tipo di indirizzo IP
 * Classifica gli IP in base al loro uso (RFC 1918 e RFC 3330)
 * 
 * IP_RESERVED: Indirizzi riservati (loopback 127.x.x.x, multicast 224-239, etc.)
 * IP_PRIVATE:  Indirizzi privati RFC 1918 (10.x.x.x, 172.16-31.x.x, 192.168.x.x)
 * IP_PUBLIC:   Indirizzi pubblici routabili su Internet
 */
typedef enum {
    IP_RESERVED,    // = 0 (assegnato automaticamente)
    IP_PRIVATE,     // = 1
    IP_PUBLIC       // = 2
} IPType;

/**
 * Struttura principale per rappresentare un indirizzo IP
 * Contiene sia la rappresentazione numerica che testuale,
 * oltre alle informazioni di classificazione e rete
 * 
 * Dimensione stimata: ~52-56 byte (con padding del compilatore)
 */
typedef struct {
    // === SEZIONE 1: Rappresentazione dell'IP ===
    
    uint32_t ip_num;              // IP in formato numerico (network byte order)
                                  // Es: 192.168.1.1 = 0xC0A80101 = 3232235777
    
    char ip_str[16];              // IP in formato stringa "xxx.xxx.xxx.xxx"
                                  // Buffer di 16 byte per "255.255.255.255\0"
    
    uint8_t octets[4];            // Singoli ottetti per accesso rapido
                                  // Es: [192, 168, 1, 1]
    
    // === SEZIONE 2: Informazioni di Rete ===
    
    uint32_t netmask;             // Netmask in formato numerico
                                  // Es: 0xFFFFFF00 = 255.255.255.0
    
    uint8_t cidr_prefix;          // Lunghezza prefisso CIDR (0-32)
                                  // Es: 24 per /24
    
    uint32_t network_addr;        // Indirizzo di rete (primo IP della subnet)
                                  // Es: 192.168.1.0
    
    uint32_t broadcast_addr;      // Indirizzo di broadcast (ultimo IP)
                                  // Es: 192.168.1.255
    
    // === SEZIONE 3: Classificazione ===
    
    char class;                   // Classe IP: 'A', 'B', 'C', 'D', 'E'
    
    IPType type;                  // Tipo: RESERVED, PRIVATE, PUBLIC
    
    // === SEZIONE 4: Metadati ===
    
    bool is_valid;                // Flag di validità dell'indirizzo
} IPAddress;

/**
 * Opzioni del programma parsate dalla command line
 * Questa struttura contiene tutti i parametri specificati dall'utente
 */
typedef struct {
    int count;                    // Numero di IP da generare (-n)
                                  // Default: 1
    
    char class_filter;            // Filtro per classe (-c)
                                  // Valori: 'A', 'B', 'C', 'D', 'E', oppure '\0' se non specificato
    
    bool has_cidr;                // Flag: è stato specificato -i?
    uint32_t cidr_network;        // Network address del CIDR (se has_cidr == true)
    uint8_t cidr_prefix;          // Lunghezza prefisso CIDR (se has_cidr == true)
    
    bool has_range;               // Flag: è stato specificato -r?
    uint32_t range_start;         // Inizio range (se has_range == true)
    uint32_t range_end;           // Fine range (se has_range == true)
} ProgramOptions;

#endif // IP_TYPES_H
EOF

echo "✅ File ip_types.h creato con successo!"
echo ""
echo "📊 Statistiche del file:"
wc -l ip_types.h
ls -lh ip_types.h

## ✅ Parte 7: Verifica e Test della Struttura

Ora creiamo un programma di test per verificare che le nostre definizioni funzionino correttamente.

### Cosa testeremo:

1. **Compilazione**: Il file compila senza errori?
2. **sizeof()**: Quanto spazio occupano le strutture?
3. **Inizializzazione**: Possiamo creare e popolare le strutture?
4. **Enum**: I valori enum sono corretti?

---

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/01

# Creiamo un programma di test
cat > test_types.c << 'EOF'
#include <stdio.h>
#include "ip_types.h"

int main() {
    printf("=== TEST STRUTTURE DATI IP ===\n\n");
    
    // Test 1: Dimensioni delle strutture
    printf("📏 DIMENSIONI STRUTTURE:\n");
    printf("   sizeof(IPType) = %zu byte\n", sizeof(IPType));
    printf("   sizeof(IPAddress) = %zu byte\n", sizeof(IPAddress));
    printf("   sizeof(ProgramOptions) = %zu byte\n\n", sizeof(ProgramOptions));
    
    // Test 2: Dimensioni dei campi individuali
    printf("🔍 DIMENSIONI CAMPI DI IPAddress:\n");
    printf("   uint32_t ip_num: %zu byte\n", sizeof(uint32_t));
    printf("   char ip_str[16]: %zu byte\n", sizeof(((IPAddress*)0)->ip_str));
    printf("   uint8_t octets[4]: %zu byte\n", sizeof(((IPAddress*)0)->octets));
    printf("   uint32_t netmask: %zu byte\n", sizeof(uint32_t));
    printf("   uint8_t cidr_prefix: %zu byte\n", sizeof(uint8_t));
    printf("   char class: %zu byte\n", sizeof(char));
    printf("   IPType type: %zu byte\n", sizeof(IPType));
    printf("   bool is_valid: %zu byte\n\n", sizeof(bool));
    
    // Test 3: Valori enum
    printf("🎨 VALORI ENUM IPType:\n");
    printf("   IP_RESERVED = %d\n", IP_RESERVED);
    printf("   IP_PRIVATE = %d\n", IP_PRIVATE);
    printf("   IP_PUBLIC = %d\n\n", IP_PUBLIC);
    
    // Test 4: Creazione di una struttura IPAddress
    printf("🏗️  CREAZIONE STRUTTURA IPAddress:\n");
    IPAddress ip = {
        .ip_num = 0xC0A80101,        // 192.168.1.1
        .ip_str = "192.168.1.1",
        .octets = {192, 168, 1, 1},
        .netmask = 0xFFFFFF00,       // 255.255.255.0
        .cidr_prefix = 24,
        .network_addr = 0xC0A80100,  // 192.168.1.0
        .broadcast_addr = 0xC0A801FF,// 192.168.1.255
        .class = 'C',
        .type = IP_PRIVATE,
        .is_valid = true
    };
    
    printf("   IP creato: %s\n", ip.ip_str);
    printf("   IP numerico: 0x%08X\n", ip.ip_num);
    printf("   Ottetti: [%d, %d, %d, %d]\n", 
           ip.octets[0], ip.octets[1], ip.octets[2], ip.octets[3]);
    printf("   Classe: %c\n", ip.class);
    printf("   Tipo: %d (0=Ris, 1=Pri, 2=Pub)\n", ip.type);
    printf("   CIDR: /%d\n", ip.cidr_prefix);
    printf("   Valido: %s\n\n", ip.is_valid ? "Sì" : "No");
    
    // Test 5: Creazione struttura ProgramOptions
    printf("⚙️  CREAZIONE STRUTTURA ProgramOptions:\n");
    ProgramOptions opts = {
        .count = 10,
        .class_filter = 'A',
        .has_cidr = true,
        .cidr_network = 0x0A000000,  // 10.0.0.0
        .cidr_prefix = 8,
        .has_range = false,
        .range_start = 0,
        .range_end = 0
    };
    
    printf("   Opzioni parsate:\n");
    printf("   - Numero IP: %d\n", opts.count);
    printf("   - Filtro classe: %c\n", opts.class_filter);
    printf("   - Ha CIDR: %s\n", opts.has_cidr ? "Sì" : "No");
    printf("   - CIDR prefix: /%d\n", opts.cidr_prefix);
    printf("   - Ha range: %s\n\n", opts.has_range ? "Sì" : "No");
    
    printf("✅ Tutti i test completati con successo!\n");
    
    return 0;
}
EOF

# Compiliamo
echo "🔨 Compilazione in corso..."
gcc -std=c11 -Wall -Wextra -o test_types test_types.c

if [ $? -eq 0 ]; then
    echo "✅ Compilazione riuscita!\n"
    echo "▶️  Esecuzione del programma:\n"
    echo "═══════════════════════════════════════════════════════════"
    ./test_types
    echo "═══════════════════════════════════════════════════════════"
else
    echo "❌ Errore di compilazione!"
    exit 1
fi

## 🔬 Parte 8: Analisi dei Risultati

### Cosa abbiamo osservato?

1. **sizeof(IPAddress)**: Probabilmente 52-60 byte
   - Somma campi: ~43 byte
   - Padding: ~9-17 byte
   - Il compilatore aggiunge padding per allineamento

2. **sizeof(ProgramOptions)**: Circa 20-28 byte
   - Dipende dall'allineamento di bool e int

3. **Valori enum**: 0, 1, 2
   - Assegnazione automatica corretta

### Perché il padding?

Esempio concreto da IPAddress:

```
uint8_t cidr_prefix;   // 1 byte a offset 24
[padding: 3 byte]      // Aggiunto dal compilatore
uint32_t network_addr; // 4 byte a offset 28 (allineato!)
```

Il compilatore inserisce 3 byte di padding dopo `cidr_prefix` per garantire che `network_addr` (uint32_t) inizi a un indirizzo multiplo di 4.

### Come verificare il padding esatto?

```c
#include <stddef.h>

// Offset di un campo
size_t offset = offsetof(IPAddress, network_addr);
printf("network_addr è a offset %zu\n", offset);
```

---

## 🎯 Parte 9: Best Practices e Pattern Comuni

### 1. Inizializzazione Zero

```c
// BENE: inizializza tutto a zero
IPAddress ip = {0};

// MALE: campi non inizializzati contengono garbage
IPAddress ip;
printf("%d\n", ip.ip_num); // ⚠️ Valore indefinito!
```

### 2. Designated Initializers (C99+)

```c
// Inizializza solo i campi necessari, il resto a zero
IPAddress ip = {
    .ip_num = 0xC0A80101,
    .class = 'C',
    .is_valid = true
    // Altri campi automaticamente = 0
};
```

### 3. Const Correctness

```c
// Funzione che legge ma non modifica
void print_ip(const IPAddress *ip) {
    printf("%s\n", ip->ip_str);
    // ip->class = 'A'; // ❌ Errore di compilazione!
}
```

### 4. Sentinel Values

```c
// Usa is_valid per indicare errori
IPAddress parse_ip(const char *str) {
    IPAddress result = {0};
    
    if (/* parsing fallito */) {
        result.is_valid = false;
        return result;  // Ritorna struct "invalida"
    }
    
    // ... popolamento ...
    result.is_valid = true;
    return result;
}

// Uso
IPAddress ip = parse_ip("192.168.1.1");
if (!ip.is_valid) {
    fprintf(stderr, "Errore parsing\n");
    return 1;
}
```

### 5. Typedef per Leggibilità

```c
// Senza typedef
enum IPType { ... };
enum IPType type = IP_PRIVATE;  // Verboso

// Con typedef
typedef enum { ... } IPType;
IPType type = IP_PRIVATE;       // Pulito
```

---

## 🧪 Parte 10: Esercizi Pratici

### Esercizio 1: Aggiungi un campo

Modifica `IPAddress` per aggiungere un campo `uint16_t port` (per memorizzare una porta TCP/UDP).

**Domande:**
1. Dove lo posizioni per minimizzare il padding?
2. Come cambia il `sizeof(IPAddress)`?

### Esercizio 2: Crea un nuovo enum

Crea un enum `IPVersion` con valori `IPV4` e `IPV6`.

### Esercizio 3: Calcola il padding

Data questa struct:

```c
struct Mystery {
    char a;
    uint32_t b;
    uint16_t c;
    char d;
};
```

Disegna il layout in memoria e calcola `sizeof(struct Mystery)` su architettura a 64 bit.

### Esercizio 4: Ottimizza una struct

Riordina i campi di `struct Mystery` per minimizzare il padding.

---

## 📚 Parte 11: Recap e Prossimi Passi

### ✅ Cosa abbiamo imparato

1. **Tipi a larghezza fissa**: `uint32_t`, `uint8_t` per portabilità
2. **Enumerazioni**: valori semantici type-safe
3. **Strutture complesse**: composizione di più tipi
4. **Layout di memoria**: allineamento e padding
5. **Header guards**: prevenire inclusioni multiple
6. **Best practices**: inizializzazione, const, sentinel values

### 🎯 Obiettivi raggiunti

- ✅ Definito `IPType` enum
- ✅ Definito `IPAddress` struct (47 campi, ~52 byte)
- ✅ Definito `ProgramOptions` struct
- ✅ Creato `ip_types.h` completo
- ✅ Testato compilazione e utilizzo

### 🔗 Integrazione con il progetto

`ip_types.h` è ora la **base** del progetto. Tutti i moduli successivi lo includeranno:

```c
#include "ip_types.h"  // In TUTTI i file
```

### ➡️ Prossimo Notebook: Conversioni IP

Nel **Notebook 2** implementeremo le funzioni per:
- Convertire stringa → uint32_t (`string_to_ip`)
- Convertire uint32_t → stringa (`ip_to_string`)
- Estrarre ottetti da IP
- Comporre IP da ottetti

Useremo operazioni **bitwise** (shift, mask, OR, AND) per manipolare gli IP a livello di bit.

---

### 🎓 Concetti chiave da ricordare

```
┌─────────────────────────────────────────────────────────────┐
│  1. Un IP è 32 bit → uint32_t                               │
│  2. Un ottetto è 8 bit → uint8_t                            │
│  3. Enum = valori con significato semantico                 │
│  4. Struct = raggruppamento logico di dati correlati        │
│  5. Padding = byte aggiunti per allineamento                │
│  6. Header guards = #ifndef/#define/#endif                  │
│  7. Sempre inizializzare le struct (almeno = {0})           │
└─────────────────────────────────────────────────────────────┘
```

---

## 🎉 Complimenti!

Hai completato il primo notebook! Ora hai una solida comprensione delle strutture dati del progetto RANDIP.

**File creato**: `ip_types.h` (base per tutto il progetto)  
**Righe di codice**: ~100 linee di definizioni  
**Concetti C**: 6+ nuovi concetti padroneggiati

Procedi al **Notebook 2** per iniziare con le conversioni IP! 🚀